In [1]:
import pandas as pd

In [24]:
import openai

In [18]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [27]:
import openai
import os

from dotenv import load_dotenv

load_dotenv()

OPENAI_KEY = os.getenv('OPENAI_API_KEY')

openai.api_key = OPENAI_KEY

Data Generation of Students

In [2]:
# Random data generation
data = {
    "Name": ["Alice Smith", "Bob Johnson", "Charlie Brown", "Dana Lee"],
    "Field of study": ["Computer Science", "Mechanical Engineering", "Business Administration", "Biochemistry"],
    "Year of study": ["2nd Year", "3rd Year", "1st Year", "2nd Year"],
    "List of subjects": [
        "Algorithms;Data Structures;Machine Learning;Databases;Operating Systems",
        "Statics;Dynamics;Thermodynamics;Fluid Mechanics;Materials Science",
        "Marketing;Finance;Accounting;Economics;Management",
        "Organic Chemistry;Biology;Genetics;Microbiology;Biochemistry"
    ],
    "Preferred Learning Styles": [
        "Visual;Kinesthetic",
        "Auditory;Kinesthetic",
        "Visual;Auditory",
        "Visual;Kinesthetic"
    ],
    "Personal Objectives": [
        "Prepare for Machine Learning certification exam",
        "Improve grades in Thermodynamics;Prepare for senior design project",
        "Gain a summer internship;Improve presentation skills",
        "Prepare for MCAT;Improve lab skills"
    ],
    "Challenges": [
        "Difficulty in understanding complex algorithms;Managing study time effectively",
        "Struggles with complex theoretical concepts;Time management issues",
        "Difficulty with finance concepts;Public speaking anxiety",
        "Struggles with complex biochemical pathways;Balancing study with lab work"
    ],
    "Extracurricular activities": [
        "Member of the coding club;Volunteer at local community center",
        "President of the engineering club;Intern at an automotive company",
        "Member of the debate team;Volunteer at local non-profit",
        "Assistant in a research lab;Member of the biochemistry student association"
    ]
}

In [3]:
df = pd.DataFrame(data)
output_csv = 'students_data.csv'
df.to_csv(output_csv, index=False)
print("Data Saved in csv file")

Data Saved in csv file


Prompt Generation

In [7]:
prompt_template = """
You are an advanced educational assistant tasked with creating a personalized study timetable for a student. You have access to a range of information about the student, including their name, field of study, year of study, list of subjects, preferred learning styles, personal objectives, challenges, and extracurricular activities. Use the provided data to generate a detailed study timetable that addresses the student’s academic requirements and aligns with their unique needs and aspirations. Ensure the timetable is structured and comprehensive, taking into account all the provided information.

Student Information:
- Name: {name}
- Field of Study: {field_of_study}
- Year of Study: {year_of_study}
- List of Subjects: {list_of_subjects}
- Preferred Learning Styles: {preferred_learning_styles}
- Personal Objectives: {personal_objectives}
- Challenges: {challenges}
- Extracurricular Activities: {extracurricular_activities}

Structure of the Study Timetable:
1. Introduction
   - Brief overview of the student’s academic background and goals.
   - Summary of the student’s preferred learning styles and how they will be incorporated into the timetable.

2. Academic Schedule
   - Detailed weekly schedule for each subject, including study hours, revision sessions, and assessment preparation.
   - Allocation of study time based on the student’s strengths and areas that need improvement.

3. Learning Techniques
   - Specific learning techniques tailored to the student’s preferred learning styles (visual, auditory, kinesthetic).
   - Use of multimedia resources, interactive sessions, practical exercises, and group studies as applicable.

4. Personal Objectives and Challenges
   - Customized strategies to achieve the student’s personal objectives (e.g., preparing for a specific exam, mastering a particular subject).
   - Practical solutions and support mechanisms to overcome the student’s challenges (e.g., dealing with a learning difficulty, managing time effectively).

5. Extracurricular Integration
   - Balancing academic responsibilities with extracurricular activities.
   - Incorporating time management skills and stress-relief practices to maintain a healthy academic-extracurricular balance.

6. Assessment and Feedback
   - Regular self-assessment techniques to monitor progress.
   - Scheduled feedback sessions with tutors or mentors to review performance and make necessary adjustments to the timetable.

7. Resources and Support
   - List of recommended resources (books, online courses, study groups, etc.).
   - Contact information for academic support services, peer study groups, and other helpful resources.
"""

In [8]:
input_csv = 'students_data.csv'
students_df = pd.read_csv(input_csv)

In [10]:
generated_prompts = []

In [11]:
for index, row in students_df.iterrows():
    # Fill the prompt with the student's data
    filled_prompt = prompt_template.format(
        name=row['Name'],
        field_of_study=row['Field of study'],
        year_of_study=row['Year of study'],
        list_of_subjects=row['List of subjects'],
        preferred_learning_styles=row['Preferred Learning Styles'],
        personal_objectives=row['Personal Objectives'],
        challenges=row['Challenges'],
        extracurricular_activities=row['Extracurricular activities']
    )

    # Store the prompt
    generated_prompts.append({
        'Name': row['Name'],
        'Field of Study': row['Field of study'],
        'Year of Study': row['Year of study'],
        'Generated Prompt': filled_prompt
    })

In [12]:
output_df = pd.DataFrame(generated_prompts)

In [14]:
output_csv = 'generated_prompts.csv'
output_df.to_csv(output_csv, index=False)
print(f"Generated prompts have been saved to {output_csv}")

Generated prompts have been saved to generated_prompts.csv


Giving prompts to generate output

In [15]:
prompts_df = pd.read_csv(output_csv)

In [29]:
study_plans = []

for index, row in prompts_df.iterrows():
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",  
        prompt=row['Generated Prompt'],
        max_tokens=1500, 
        temperature=0.7
    )
    study_plan = response.choices[0].text.strip()

    study_plans.append({
        'Name': row['Name'],
        'Field of Study': row['Field of Study'],
        'Year of Study': row['Year of Study'],
        'Generated Study Plan': study_plan
    })

In [30]:
study_plans_df = pd.DataFrame(study_plans)

In [31]:
study_plans_csv = 'generated_study_plans.csv'
study_plans_df.to_csv(study_plans_csv, index=False)
print(f"Generated study plans have been saved to {study_plans_csv}")

Generated study plans have been saved to generated_study_plans.csv


In [32]:
for plan in study_plans:
    print(f"Name: {plan['Name']}\n")
    print(f"Field of Study: {plan['Field of Study']}\n")
    print(f"Year of Study: {plan['Year of Study']}\n")
    print(f"Generated Study Plan:\n{plan['Generated Study Plan']}\n")

Name: Alice Smith

Field of Study: Computer Science

Year of Study: 2nd Year

Generated Study Plan:
8. Conclusion
   - Summary of the study timetable and its alignment with the student’s unique needs and aspirations.
   - Encouragement for the student to follow the timetable consistently and make necessary adjustments as needed.

Name: Bob Johnson

Field of Study: Mechanical Engineering

Year of Study: 3rd Year

Generated Study Plan:
8. Conclusion
   - Recap of the study timetable and its alignment with the student’s academic requirements and personal goals.
   - Encouragement and motivation for the student to follow the timetable consistently and seek help when needed.

Name: Charlie Brown

Field of Study: Business Administration

Year of Study: 1st Year

Generated Study Plan:
- Encouragement to seek help when needed and to utilize available resources effectively.

Conclusion:
   - Summary of the study timetable and its alignment with the student’s academic requirements and unique nee

Or you can check the output here

https://chatgpt.com/share/af623308-d5ad-4fc7-a6c1-594bc5502c18